In [3]:
# Importing the libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from collections import Counter
import unicodedata

# Reading our data
df = pd.read_csv('disease.csv')
df["Target"]= df["Target"].str.replace("\t", ",", case = False) 
df.head()

C:\Users\obero\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\obero\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\obero\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.txa6yqsd3gcqqc22geq54j2udcxdxhwn.gfortran-win_amd64.dll
  stacklevel=1)


,Target,Source,Medicine
0,"drool,agitation,nightmare,rhonchus,consciousne...",Alzheimer's disease,"rivastigmine,donepezil,galantamine"
1,"fever,night sweat,spontaneous rupture of membr...",HIV,"abacavir,lamivudine,stavudine"
2,"yellow sputum,cachexia,chill,decreased body we...",Pneumocystis carinii pneumonia,"tetracyclines,macrodile antibiotic,penicillin"
3,"dysarthria,asthenia,speech slurred,facial pare...",accident cerebrovascular,"statin,alteplase,apixaban"
4,"fever,night sweat,spontaneous rupture of membr...",acquired immuno-deficiency syndrome,"amantadine,acyclovir,rolaids"


In [4]:
# Text Preprocessing
list1 = []
corpus = []
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text
for i in range(149):
    list1.append(remove_accented_chars(df['Target'][i]))
df['Target'] = list1


for i in range(0, 149):
    review = re.sub('[^a-zA-Z]', ' ', df['Target'][i])
    review = review.lower()
    review = review.split()
    #ps = PorterStemmer() 
    #review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = [word for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
    
    
# Making a dictionary of disease and its symptoms
dict1 = {}
for i in range(149):
    dict1[df['Source'][i]] = corpus[i]

In [ ]:
# Chat with user
while True:
    symptoms = input('Tell me the symptoms or type quit to exit:')
    symptoms = symptoms.lower()
    if symptoms == 'quit':
        print('Bye! Have a good day!')
        break
    else:
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~''' 
        for x in symptoms.lower(): 
            if x in punctuations: 
                symptoms = symptoms.replace(x, " ") 
        symptoms = symptoms.split()
        list1 = []
        for i in symptoms:
            for k,v in dict1.items():
                if i in v:
                    k = k.replace(u'\xa0', u' ')
                    list1.append([k])
                else:
                    pass
        word_counts = Counter(word for words in list1 for word in words)
        print('The disease(s) you are most likely to have on the basis of the symptoms given are:  ')
        count = 1
        for elements in word_counts.most_common(5):
            print('\n{}.'.format(count),elements[0])
            print('\nThe medicines/drugs for it are: ')
            for items in df[df['Source'] == elements[0]]['Medicine']:
                items = items.split(',')
                for medicine in items:
                    print(medicine.title())
            count += 1

# Proper NLP

In [ ]:
from newspaper import Article
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings

#Ignore any warning messages
warnings.filterwarnings('ignore')

#Get the article URL
article = Article('https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521')
article.download()
article.parse()
article.nlp()
corpus = article.text

In [ ]:
#Tokenization
text = corpus
sent_tokens = nltk.sent_tokenize(text) #Convert the text into a list of sentences

In [ ]:
GREETING_INPUTS = ["hi", "hello", "hola", "greetings", "wassup", "hey"]

#Greeting responses back to the user
GREETING_RESPONSES=["howdy", "hi", "hey", "what's good", "hello", "hey there"]

#Function to return a random greeting response to a users greeting
def greeting(sentence):
    #if the user's input is a greeting, then return a randomly chosen greeting response
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
remove_punct_dict = dict(  ( ord(punct),None) for punct in string.punctuation)
def LemNormalize(text):
    return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

In [ ]:
def response(user_response):
    #The users response / query
    #user_response = 'What is chronic kidney disease'

    user_response = user_response.lower() #Make the response lower case

    ###Print the users query/ response
    #print(user_response)

    #Set the chatbot response to an empty string
    robo_response = ''

    #Append the users response to the sentence list
    sent_tokens.append(user_response)

    ###Print the sentence list after appending the users response
    #print(sent_tokens)

    #Create a TfidfVectorizer Object
    TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words='english')

    #Convert the text to a matrix of TF-IDF features
    tfidf = TfidfVec.fit_transform(sent_tokens)

    ###Print the TFIDF features
    #print(tfidf)

    #Get the measure of similarity (similarity scores)
    vals = cosine_similarity(tfidf[-1], tfidf)

    #Print the similarity scores
    #print(vals)

    #Get the index of the most similar text/sentence to the users response
    idx = vals.argsort()[0][-2]

    #Reduce the dimensionality of vals
    flat = vals.flatten()

    #sort the list in ascending order
    flat.sort()

    #Get the most similar score to the users response
    score = flat[-2]

    #Print the similarity score
    #print(score)

    #If the variable 'score' is 0 then their is no text similar to the users response
    if(score == 0):
        robo_response = robo_response+"I apologize, I don't understand."
    else:
        robo_response = robo_response+sent_tokens[idx]
  
    #Print the chat bot response
    #print(robo_response)
  
    #Remove the users response from the sentence tokens list
    sent_tokens.remove(user_response)
  
    return robo_response

In [ ]:
print("HealthBot: I am HealthBot. If you want to exit, type Bye!")
flag = True
while(flag == True):
    user_response = input('User Input: ')
    user_response = user_response.lower()
    if(user_response != 'bye'):
        if(user_response == 'thanks' or user_response =='thank you'):
            flag=False
            print("HealthBot: You are welcome !")
        else:
            if(greeting(user_response) != None):
                print("HealthBot: "+greeting(user_response))
            else:
                print("HealthBot: "+response(user_response))       
    else:
        flag = False
        print("HealthBot: Chat with you later !")